# End-End Machine Learning Project: Random Forest Classifier

## 1. Frame the Problem

### Questions to Ask

1. What is the **Business Objective** in Building this Machine Learning Model?
    - **Goal:** To Detect Fradualent Transactions while Minimizing False Negatives
  
2. How does the company expect to use and benefit from this model?
    - Suppose in this seanario the company is a financial institution such as a Bank or Federal Credit Union, they would use the model to prematurely detect a fradualent transaction as-it is attempted to be processed on the consumer side (when a 'customer' is making a purchase in-store). The actual cardholder would then file a claim in a attempt to be reimbursed which most financial institutions usually do, but this results in **$12.5 billion** from credit card fraud in the U.S. in 2025, and **$117 million** from debit card fraud in 2025, [Merchant Cost Consulting](https://merchantcostconsulting.com/lower-credit-card-processing-fees/credit-card-fraud-statistics/). If the model is implmented by SWEs in these instiutions properly it can save billions of dollars per year.

3. Performance Metrics Used
    - Primarily **Percision-Recall AUC**, which is more important than **accuracy** for rare-event detection.
  
4. Assumptions Made
    1. Historical Data is Representative
       - The fradulent transaction data roughly reflects future fraudulent transaction data
    2. Labels are Mostly Correct
       - As we find later the labels need to be converted from float to int, but this is a very minor issue
    3. Each Row is a Independent Observation of a Potentially Fraudulent Transaction
    4. Fraud Detection is a Binary Classification Task (1 - True / 0 - False )

## 2. Get the Data

At this stage we are only **loading** and **inspecting** the data. No data cleaning or transformations, yet.

In [1]:
import pandas as pd

card_data = pd.read_csv("./datasets/card_transdata.csv")

### Display / Explore the  Data

In [2]:
card_data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


In [3]:
card_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   distance_from_home              1000000 non-null  float64
 1   distance_from_last_transaction  1000000 non-null  float64
 2   ratio_to_median_purchase_price  1000000 non-null  float64
 3   repeat_retailer                 1000000 non-null  float64
 4   used_chip                       1000000 non-null  float64
 5   used_pin_number                 1000000 non-null  float64
 6   online_order                    1000000 non-null  float64
 7   fraud                           1000000 non-null  float64
dtypes: float64(8)
memory usage: 61.0 MB


In [4]:
card_data['distance_from_last_transaction'].value_counts()

distance_from_last_transaction
0.311140    1
0.175592    1
0.805153    1
5.600044    1
0.566486    1
           ..
0.112651    1
2.683904    1
1.472687    1
0.242023    1
0.318110    1
Name: count, Length: 1000000, dtype: int64

In [5]:
card_data.describe(include="all")

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,26.628792,5.036519,1.824182,0.881536,0.350399,0.100608,0.650552,0.087403
std,65.390784,25.843093,2.799589,0.323157,0.477095,0.300809,0.476796,0.282425
min,0.004874,0.000118,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.878008,0.296671,0.475673,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.967760,0.998650,0.997717,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.743985,3.355748,2.096370,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,11851.104565,267.802942,1.000000,1.000000,1.000000,1.000000,1.000000


#### Data Exploration

In [6]:
card_data.value_counts(normalize=True)

distance_from_home  distance_from_last_transaction  ratio_to_median_purchase_price  repeat_retailer  used_chip  used_pin_number  online_order  fraud
57.877857           0.311140                        1.945940                        1.0              1.0        0.0              0.0           0.0      0.000001
10.829943           0.175592                        1.294219                        1.0              0.0        0.0              0.0           0.0      0.000001
5.091079            0.805153                        0.427715                        1.0              0.0        0.0              1.0           0.0      0.000001
2.247564            5.600044                        0.362663                        1.0              1.0        0.0              1.0           0.0      0.000001
44.190936           0.566486                        2.222767                        1.0              1.0        0.0              1.0           0.0      0.000001
                                              

### Clean the Data

This is to check for any NaN (i.e. null values) before performing the **training, validation, and test split**. **An error will be thrown** if __NaN__ values are **present**.

In [7]:
card_data.dropna(how="any")

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
999995,2.207101,0.112651,1.626798,1.0,1.0,0.0,0.0,0.0
999996,19.872726,2.683904,2.778303,1.0,1.0,0.0,0.0,0.0
999997,2.914857,1.472687,0.218075,1.0,1.0,0.0,1.0,0.0
999998,4.258729,0.242023,0.475822,1.0,0.0,0.0,1.0,0.0


This is here because the labels in this dataset were floating points rather than 1's or 0's which is important for **binary classification** tasks. The floats were converted to integers in the second line.

In [8]:
import numpy as np

print(np.unique(card_data['fraud']))

[0. 1.]


This is considered *label normalizaton* versus *feature preprocessing*, no data pipelines needed here

In [9]:
card_data['fraud'] = card_data['fraud'].astype(int)

### Creating a Test Set

#### Perform Stratified Sampling when Creating the Test Set

#### Why Stratified Sampling versus Random Sampling?
- The dataset is **large** in this case so random sampling **could** work, but to be sure to avoid introducting data bias (or when using smaller datasets) **stratified sampling**. **Stratified Sampling** is when you split the dataset into homogenous subgroups known as *strata* and then the right amount samples are then gathered from each subgroup(strata) to ensure that test set is farily representative of all transactions.


In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

card_data_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in card_data_split.split(card_data, card_data['fraud']):
    train_set = card_data.loc[train_idx]
    test_set = card_data.iloc[test_idx]

X_train = train_set.drop('fraud', axis=1)
y_train = train_set['fraud']

X_test = test_set.drop('fraud', axis=1)
y_test = test_set['fraud']

In [11]:
display(X_train)

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order
950236,3.426737,7.234169,0.208383,1.0,1.0,0.0,1.0
358251,15.106926,0.867626,2.713531,1.0,1.0,0.0,1.0
597434,31.797354,3.159440,0.304588,1.0,0.0,0.0,1.0
296537,10.214106,0.034442,0.340111,1.0,0.0,0.0,0.0
235768,81.804301,4.735865,1.284013,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
62473,18.062687,1.129769,1.039432,1.0,1.0,0.0,0.0
328239,8.350774,0.289603,2.004832,1.0,0.0,0.0,1.0
467315,4.529391,45.922502,1.831249,1.0,1.0,0.0,1.0
911151,19.110160,0.439835,0.166790,1.0,0.0,1.0,0.0


## Data Preprocessing (Transformation) Pipelines


In [12]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

num_features = X_train.select_dtypes(include=["int64", "float64"]).columns
#cat_features = X_train.select_dtypes(include=["object", "category"]).columns

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

#cat_pipeline = Pipeline([
#    ("imputer", SimpleImputer(strategy="most_frequent")),
#    ("onehot", OneHotEncoder(handle_unknown="ignore"))
#])

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_features),
    # ("cat", cat_pipeline, cat_features)
])

__***Note**__: That the categorical features are commented out because there is __no__ categorical data in this dataset, if using another dataset that __does__ have categorical data then the commetted out code would be used.

## Select and Train a Model

### Create a Baseline Model

This is important because it helps establish the minimum acceptable performance.

In [13]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

baseline = Pipeline([
    ("prep", preprocessing),
    ("model", DummyClassifier(strategy="most_frequent"))
])

baseline_scores = cross_val_score(
    baseline,
    X_train,
    y_train,
    scoring="average_precision",
    cv=5
)

baseline_scores.mean()

np.float64(0.08740250000000001)

### Train and Evaluate the RandomForest Classifier Model

In [14]:
from sklearn.ensemble import RandomForestClassifier

fraud_classifier = make_pipeline(preprocessing, RandomForestClassifier(random_state=42))
fraud_classifier.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('randomforestclassifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transfo

### Better Evaluation with Cross-Validation

As you may have seen previously in the baseline model this is used to evaluate and get the Precision-Recall AUC Performance Metric

In [15]:
frf_precision_recall_auc = cross_val_score(
    fraud_classifier,
    X_train,
    y_train,
    scoring="average_precision",
    cv=5
)

In [16]:
frf_precision_recall_auc.mean()

np.float64(0.9999999887510533)

## Fine Tune the Model 
### Use Grid Search For Hyper-Parameter Tuning 

In [17]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestClassifier(random_state=42)),
])

In [ ]:
# What regular grid search looks like
'''from sklearn.model_selection import GridSearchCV
param_grid = {
    #  Preprocessing hyperparameters
    "preprocessing__num__imputer__strategy": ["mean", "median"],

    # Random Forest hyperparameters
    "random_forest__n_estimators": [50, 100, 200, 300],
    "random_forest__max_depth": [4, 8, None],
    "random_forest__max_features": [2, 3, 4],
    "random_forest__max_leaf_nodes": [4, 6, None],
    "random_forest__min_samples_leaf": [1, 2, 4],
    "random_forest__min_samples_split": [5, 10]
}

# Initialize the GridSearchCV object
grid_search = GridSearchCV(full_pipeline, param_grid, cv=5, scoring='average_precision', n_jobs=-1, verbose=3)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score achieved during the grid search
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)'''

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


# Randomized Grid Search

In [23]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {
        "preprocessing__num__imputer__strategy": ["mean", "median"],
        "random_forest__n_estimators": randint(low=50, high=300),
        "random_forest__max_depth": randint(low=1, high=8),
        "random_forest__max_features": randint(low=2, high=4),
        "random_forest__max_leaf_nodes": randint(low=2, high=8),
        "random_forest__min_samples_leaf": randint(low=1, high=6),
        "random_forest__min_samples_split": randint(low=5, high=10),
}

rnd_search = RandomizedSearchCV(
    full_pipeline, param_distributions=param_distribs, n_iter=10, cv=5,
    scoring='average_precision', n_jobs=4, verbose=3, random_state=42)

rnd_search.fit(X_train, y_train)

[CV 5/5] END preprocessing__num__imputer__strategy=mean, random_forest__max_depth=2, random_forest__max_features=2, random_forest__max_leaf_nodes=3, random_forest__min_samples_leaf=4, random_forest__min_samples_split=8, random_forest__n_estimators=257;, score=0.912 total time=  43.9s
[CV 5/5] END preprocessing__num__imputer__strategy=mean, random_forest__max_depth=3, random_forest__max_features=2, random_forest__max_leaf_nodes=4, random_forest__min_samples_leaf=5, random_forest__min_samples_split=9, random_forest__n_estimators=152;, score=0.961 total time=  23.0s
[CV 1/5] END preprocessing__num__imputer__strategy=median, random_forest__max_depth=5, random_forest__max_features=3, random_forest__max_leaf_nodes=3, random_forest__min_samples_leaf=4, random_forest__min_samples_split=9, random_forest__n_estimators=210;, score=0.909 total time=  41.2s
[CV 3/5] END preprocessing__num__imputer__strategy=mean, random_forest__max_depth=5, random_forest__max_features=3, random_forest__max_leaf_nod

/Users/kylew/Documents/School/Work/483/labs/lab-venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV 1/5] END preprocessing__num__imputer__strategy=mean, random_forest__max_depth=4, random_forest__max_features=2, random_forest__max_leaf_nodes=4, random_forest__min_samples_leaf=5, random_forest__min_samples_split=9, random_forest__n_estimators=152;, score=0.961 total time=  21.3s
[CV 2/5] END preprocessing__num__imputer__strategy=median, random_forest__max_depth=3, random_forest__max_features=2, random_forest__max_leaf_nodes=4, random_forest__min_samples_leaf=3, random_forest__min_samples_split=9, random_forest__n_estimators=149;, score=0.962 total time=  19.1s
[CV 2/5] END preprocessing__num__imputer__strategy=mean, random_forest__max_depth=4, random_forest__max_features=2, random_forest__max_leaf_nodes=4, random_forest__min_samples_leaf=5, random_forest__min_samples_split=9, random_forest__n_estimators=152;, score=0.959 total time=  21.4s
[CV 3/5] END preprocessing__num__imputer__strategy=median, random_forest__max_depth=3, random_forest__max_features=2, random_forest__max_leaf_n

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'preprocessing__num__imputer__strategy': ['mean', 'median'], 'random_forest__max_depth': <scipy.stats....t 0x117ba6330>, 'random_forest__max_features': <scipy.stats....t 0x117ba65a0>, 'random_forest__max_leaf_nodes': <scipy.stats....t 0x11865d280>, ...}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",10
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'average_precision'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",4
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so

In [24]:
# Print the best parameters and the best score achieved during the random grid search
print("Best parameters:", rnd_search.best_params_)
print("Best cross-validation score:", rnd_search.best_score_)

Best parameters: {'preprocessing__num__imputer__strategy': 'median', 'random_forest__max_depth': 6, 'random_forest__max_features': 2, 'random_forest__max_leaf_nodes': 5, 'random_forest__min_samples_leaf': 1, 'random_forest__min_samples_split': 5, 'random_forest__n_estimators': 268}
Best cross-validation score: 0.9915753525327442
[CV 4/5] END preprocessing__num__imputer__strategy=median, random_forest__max_depth=2, random_forest__max_features=3, random_forest__max_leaf_nodes=2, random_forest__min_samples_leaf=2, random_forest__min_samples_split=9, random_forest__n_estimators=179;, score=0.885 total time=  25.9s
[CV 3/5] END preprocessing__num__imputer__strategy=median, random_forest__max_depth=4, random_forest__max_features=2, random_forest__max_leaf_nodes=5, random_forest__min_samples_leaf=4, random_forest__min_samples_split=9, random_forest__n_estimators=57;, score=0.990 total time=   8.9s


## Finalized Random Forest Graph

In [18]:
fraud_final_rfc = make_pipeline(preprocessing, RandomForestClassifier(max_depth=6, max_features=2, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=5, n_estimators=268, random_state=42))
fraud_final_rfc.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('randomforestclassifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transfo

In [19]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

fraud_rf_tree = fraud_final_rfc[-1].estimators_[0]
class_names = [str(cls) for cls in sorted(y_train.unique())]
plt.figure(figsize=(20,10), dpi=300)
plot_tree(fraud_rf_tree, filled=True, feature_names=X_train.columns, class_names=class_names, rounded=True)
plt.title('Decision Tree from Final Random Forest Classifier')
plt.savefig("./pics/rf_final_tree.svg", format="svg", bbox_inches="tight")
plt.close()

## Analyze Best Models and their Errors

In [20]:
fraud_final_rfc.named_steps
feature_importances = rnd_search.best_estimator_["random_forest"].feature_importances_

NameError: name 'rnd_search' is not defined

In [21]:
feature_importance_df = pd.DataFrame({
    "feature": fraud_final_rfc.named_steps["columntransformer"].get_feature_names_out(),
    "importance": feature_importances
}).sort_values(by="importance", ascending=False)

feature_importance_df.head(10)

NameError: name 'feature_importances' is not defined

In [34]:
import matplotlib.pyplot as plt

top_n = 15

feature_importance_df.head(top_n).plot(
    kind="barh",
    x="feature",
    y="importance",
    legend=False,
    figsize=(8, 6)
)

plt.gca().invert_yaxis()
plt.title("Top Feature Importances (Random Forest)")
plt.xlabel("Importance")
plt.show()

NameError: name 'feature_importance_df' is not defined

## Evaluate the System on the Test Set

In [35]:
# Hard predictions
y_test_pred = fraud_final_rfc.predict(X_test)

# Probabilities (needed for ranking metrics)
y_test_proba = fraud_final_rfc.predict_proba(X_test)[:, 1]

### Confustion Matrix

In [36]:
from sklearn.metrics import confusion_matrix, classification_report

conf_matrix = confusion_matrix(y_test, y_test_pred)
class_report = classification_report(y_test, y_test_pred, target_names=class_names)
print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)

Confusion Matrix:
 [[182519      0]
 [  5297  12184]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99    182519
           1       1.00      0.70      0.82     17481

    accuracy                           0.97    200000
   macro avg       0.99      0.85      0.90    200000
weighted avg       0.97      0.97      0.97    200000



In [37]:
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Final Confusion Matrix (Stratified Sampling)')
plt.savefig("./pics/rf_final_confustion_matrix.svg")
plt.close()

## Classification Report 

In [38]:
class_report = classification_report(y_test, y_test_pred, target_names=class_names, output_dict=True)
df_final_report = pd.DataFrame(class_report).transpose()

plt.figure(figsize=(10, 5))
sns.heatmap(df_final_report.iloc[:-1, :].drop(columns=['support']), annot=True, cmap='Blues', cbar=False)
plt.title('Classification Report (Stratified Sampling)')
plt.savefig("./pics/rf_final_class_report.svg")
plt.close()


### Get the Final Percision-Recall AUC

In [41]:
from sklearn.metrics import precision_recall_curve, auc

precision, recall, _ = precision_recall_curve(y_test, y_test_proba)
pr_auc = auc(recall, precision)

print(f"Precision–Recall AUC: {pr_auc:.4f}")

Precision–Recall AUC: 0.9922


## Launch, Monitor, and Maintain the System

In [40]:
import joblib

joblib.dump(fraud_final_rfc, "./models/fraud_checker_Random_Forest.pkl")

['./models/fraud_checker_Random_Forest.pkl']